In [1]:
from utz import *
from utz import plots
from utz.colors import swatches
import plotly.express as px
import plotly.graph_objects as go

In [2]:
ymrgtb_cd = read_parquet('s3://ctbk/aggregated/ymrgtb_cd.parquet')
ymrgtb_cd

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
0,HB,2021,5,0,Customer,docked_bike,7583,12921780,2021-05-01
1,HB,2021,5,0,Subscriber,docked_bike,4204,3636441,2021-05-01
2,HB,2021,6,0,Customer,classic_bike,6112,7552060,2021-06-01
3,HB,2021,6,0,Customer,docked_bike,9348,14783466,2021-06-01
4,HB,2021,6,0,Subscriber,classic_bike,3829,2585386,2021-06-01
...,...,...,...,...,...,...,...,...,...
1107,NYC,2022,12,0,Customer,classic_bike,176993,187640589,2022-12-01
1108,NYC,2022,12,0,Customer,docked_bike,4473,11773487,2022-12-01
1109,NYC,2022,12,0,Customer,electric_bike,47847,56476131,2022-12-01
1110,NYC,2022,12,0,Subscriber,classic_bike,1201344,785446743,2022-12-01


Drop single `motivate_dockless_bike` ride

In [3]:
ymrgtb_cd.groupby('Rideable Type').Count.sum()

Rideable Type
classic_bike               41877189
docked_bike                 9572758
electric_bike               7156285
motivate_dockless_bike            1
unknown                   114176271
Name: Count, dtype: int64

In [4]:
ymrgtb_cd[ymrgtb_cd['Rideable Type'] == 'motivate_dockless_bike']

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
454,JC,2021,7,0,Customer,motivate_dockless_bike,1,665,2021-07-01


In [5]:
ymrgtb_cd = ymrgtb_cd[ymrgtb_cd['Rideable Type'] != 'motivate_dockless_bike']

## Build `ymrgtb_cd.json`
Used by [`index.tsx`](www/pages/index.tsx) plot

In [6]:
ymr_json = (
    ymrgtb_cd
    .assign(**{
        'Region': ymrgtb_cd.Region.replace('HB', 'HOB'),
        'User Type': ymrgtb_cd['User Type'].apply(lambda u: {'Customer': 'Daily', 'Subscriber': 'Annual',}[u])
    })
    .drop(columns='Month')
    .rename(columns={
        'Start Year': 'Year',
        'Start Month': 'Month',
    })
)
ymr_json

,Region,Year,Month,Gender,User Type,Rideable Type,Count,Duration
0,HOB,2021,5,0,Daily,docked_bike,7583,12921780
1,HOB,2021,5,0,Annual,docked_bike,4204,3636441
2,HOB,2021,6,0,Daily,classic_bike,6112,7552060
3,HOB,2021,6,0,Daily,docked_bike,9348,14783466
4,HOB,2021,6,0,Annual,classic_bike,3829,2585386
...,...,...,...,...,...,...,...,...
1107,NYC,2022,12,0,Daily,classic_bike,176993,187640589
1108,NYC,2022,12,0,Daily,docked_bike,4473,11773487
1109,NYC,2022,12,0,Daily,electric_bike,47847,56476131
1110,NYC,2022,12,0,Annual,classic_bike,1201344,785446743


In [7]:
ymr_json.to_json('www/public/assets/ymrgtb_cd.json', 'records')

In [8]:
renames = {
    'r': ['NYC', 'JC', 'HOB'],
    't': ['Annual', 'Daily'],
    'b': ['classic_bike', 'docked_bike', 'electric_bike', 'unknown',],
}
renames = {
    k: { v: idx for idx, v in enumerate(vs) }
    for k, vs in renames.items()
}
renames

{'r': {'NYC': 0, 'JC': 1, 'HOB': 2},
 't': {'Annual': 0, 'Daily': 1},
 'b': {'classic_bike': 0, 'docked_bike': 1, 'electric_bike': 2, 'unknown': 3}}

In [9]:
ymr_json2 = (
    ymr_json
    .rename(columns={
        'Region': 'r',
        'Year': 'y',
        'Month': 'm',
        'Gender': 'g',
        'User Type': 't',
        'Rideable Type': 'b',
        'Count': 'c',
        'Duration': 'd',
    })
)
ymr_json2 = (
    ymr_json2
    .assign(
        **{
            k: ymr_json2[k].apply(lambda v: o[v])
            for k, o in renames.items()
        },
        y=ymr_json2.y.apply(lambda y: y - 2000)
    )
)
ymr_json2

,r,y,m,g,t,b,c,d
0,2,21,5,0,1,1,7583,12921780
1,2,21,5,0,0,1,4204,3636441
2,2,21,6,0,1,0,6112,7552060
3,2,21,6,0,1,1,9348,14783466
4,2,21,6,0,0,0,3829,2585386
...,...,...,...,...,...,...,...,...
1107,0,22,12,0,1,0,176993,187640589
1108,0,22,12,0,1,1,4473,11773487
1109,0,22,12,0,1,2,47847,56476131
1110,0,22,12,0,0,0,1201344,785446743


In [10]:
ymr_json2.to_json('www/public/assets/ymrgtb_cd.short.json', 'records')